In [1]:
import torch
import numpy as np
import pandas as pd
import torchvision
from torch import nn
import torch.optim as optim
import time, glob
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device('cuda:0')

Test pre trained model

In [3]:
vgg_16_pre = torchvision.models.vgg16(weights=None, progress=True)

In [4]:
vgg_16_pre.classifier[6] = nn.Linear(4096, 1, bias=True)

In [5]:
vgg_16_pre.classifier.add_module('7', nn.Sigmoid())

In [6]:
vgg_16_pre

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
def train_model(model, dataset, criterion, optimizer, num_epochs = 2):
    since = time.time()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for imgs, labels in dataset:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(imgs)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()
            running_loss += loss.item() * imgs.size(0)

        epoch_loss = running_loss / 25000
        print(f'Epoch loss: {epoch_loss}')


In [14]:
class GetData(Dataset):
    def __init__(self, datapoints, labels):
        self.datapoints = datapoints
        self.labels = labels
    def __len__(self):
        return len(self.datapoints)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.datapoints[idx])
        img = torchvision.io.decode_image(img)
        img = torchvision.transforms.Resize((64, 64))(img)
        img = img / 255
        label = torch.Tensor([self.labels[idx]])
        return img, label

In [15]:
files = glob.glob('../../files/train/*.jpg')

In [16]:
labels = [1 if x.split('/')[-1].split('.')[0] == 'dog' else 0 for x in files]

In [39]:
data = GetData(files, labels)
train_dataloader = DataLoader(data, batch_size=64, num_workers=20, persistent_workers=True)

In [18]:
criterion = nn.BCELoss()

optimizer = optim.SGD(vgg_16_pre.parameters(), lr = 0.01)

vgg_16_pre.to(device)
train_model(vgg_16_pre, train_dataloader, criterion, optimizer)

Epoch loss: 0.6830381985855103
Epoch loss: 0.6695899661636353


Create custom pytorch model

In [112]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(2048, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [113]:
criterion = nn.BCELoss()

optimizer = optim.SGD(custom_model.parameters(), lr = 0.01)
custom_model = VGG16()
custom_model.to(device)

train_model(custom_model, train_dataloader, criterion, optimizer)

Epoch loss: 0.6996370206260681
Epoch loss: 0.6996475138854981
